# Borrower reliability research

Many people take out loans. But how many of them return the money on time?  
Can we help the bank determine whether the borrower will repay the money on time, based on data such as the client's marital status and number of children?

Let's do some research!

Input data from the customer - statistics on the solvency of customers. The results of the study will be taken into account when building a credit scoring model - a special system that assesses the ability of a potential borrower to return a loan to a bank.

Let's describe the data.

* children - number of children in the family 
* days_employed - total work expirience in days  
* dob_years - client's age in years
* education - client's education level
* education_id - educational level identifier
* family_status - marital status
* family_status_id - marital status identifier
* gender - client gender
* income_type - type of employment
* debt - whether they were in debt to repay loans or not
* total_income - monthly income
* purpose - purpose of the loan

The main steps of our project will be:
* Opening a data file and examining general information
* Data preprocessing
* Calculations and adding the results necessary for analysis
* Data analysis
* Hypothesis testing
* Formulation of main conclusions

The project is made in **Jupyter Notebook**, **Python** 3.7.8.
The project uses the **Pandas** library, as well as the **IPython** module and **PyMyStem3** module.

## Opening a data file and examining general information

In [1]:
# Import required libraries and modules.
import pandas as pd
from IPython.display import display
from pymystem3 import Mystem

# Read dataset.
data = pd.read_csv('data_project1.csv')
data.info()
print()
print('\033[1m' + 'Table 1. The first 10 lines of the original dataset')
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

Table 1. The first 10 lines of the original dataset


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Conclusion**

The original dataset consists of 21,525 rows and 12 columns. 
* two columns contain `float64` data 
* five columns contain `int64` data
* five columns contain `object` data

We have analyzed the data, now let's define the steps for data preprocessing:
1. It became clear that the columns **days_employed** and **total_income** have missing values (2174 each) that need to be processed.
2. It is worth noting that the **days_employed** column stores data in the `float64` format. This column, based on its name, stores data on the number of days during which the bank's client was employed at the time this table was created. Obviously, the number of days is a value that should be displayed as integers (`int64` format). There is an assumption that most likely the number of days of employment is calculated by a formula containing a division operator, as a result of which float numbers appear in the column. It is advisable to bring the format of these numbers to the `int64` format.
3. There are negative values in the **children** and **days_employed** columns. Such values do not make sense (probably a mistake was made during data collection). They need to be converted to positive values.
4. There are abnormal values in the **days_employed** column. So, for example, the number of days of service for the category of clients "пенсионер" ("retired") exceeds the maximum possible.
5. There are zero values in the **dob_years** column. We need to replace them with average values for each customer category (category by **income_type** column).
6. The **education** and **family_status** columns should be uniform in letter size.
7. In the columns **family_status**, **income_type**, purpose it is necessary to check for duplicates.
8. In the **total_income** column, it makes sense to display numbers in the `int64` format.

## Data preprocessing

### Missing values processing

In [2]:
# Get rid of negative values.
data['days_employed'] = data['days_employed'].abs()
data['children'] = data['children'].abs()

# It seems the days are expressed in hours.
# Let's convert hours to days.
def hours_to_days(presumably_hours):
    # A person in Russia can work for 22,265 hours or 61 years
    # in a lifetime.
    if presumably_hours > 22_265:
        return presumably_hours / 24
    return presumably_hours

# Apply the function.
data['days_employed'] = data['days_employed'].apply(hours_to_days)

# Create a list with client categories.
income_types = data.groupby('income_type').sum().index

# Replace zero values in the dob_years column with 
# average values in each client category.
for income_type in income_types:
    data_dob_mean = data.groupby('income_type')['dob_years'].mean()
    data.loc[
        data['income_type'] == income_type, 'dob_years'] = data.loc[
        data['income_type'] == income_type, 'dob_years'].replace(
        0, data_dob_mean[income_type]
    )

# After replacing the null values in the dob_years column, 
# the data type has changed.
# Return the data type int.
data['dob_years'] = data['dob_years'].astype(int)

In [3]:
# There are impossible values for the number of days
# of service in the days_employed column.
# Let's replace them.
def reduction(row):
    days_employed = row['days_employed']
    dob_years = row['dob_years']
    calc_days = (dob_years-14) * 365
    if  days_employed > calc_days:
        return calc_days
    return days_employed

# Apply the function.
data['days_employed'] = data.apply(reduction, axis=1)

# Replace the NaN-values in the income_type and 
# total_income columns.
for income_type in income_types:
    data_mean = data.groupby('income_type')['days_employed'].mean()
    data_median = data.loc[
        data['income_type'] == income_type, 'total_income'].median()
    data.loc[
        data['income_type'] == income_type, 'days_employed'] = data.loc[
        data['income_type'] == income_type, 'days_employed'].fillna(
        data_mean[income_type]
    )
    data.loc[
        data['income_type'] == income_type, 'total_income'] = data.loc[
        data['income_type'] == income_type, 'total_income'].fillna(
        data_median
    )
print('\033[1m' + 'Table 2. Number of missing values in columns')    
display(data.isna().sum())

Table 2. Number of missing values in columns


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Conclusion**

The first step was to handle the `NaN`-values in the **days_employed** and **total_income** columns.
There are 2174 missing values in each column, or 10% of the total data in those columns.
10% is a significant proportion, therefore, in order to maintain the objectivity of the study's conclusions, we decide not to delete this data.
`NaN`-values are present in the **days_employed** and **total_income** columns for the same clients. The reason why this data is probably omitted is that the bank does not have work books (in Russia, this is a document that indicates the labor history of a person) or 2-NDFL certificate (In Russia, this is a certificate that reflects taxes on the income of an individual) for customer data.

To understand what values you need to replace the missing values in the column with, you need to bring all anomalies into a representative form.
1. Using the `abs()` method, negative values were converted to positive values.
2. Impossibly large values of days of service were found for some clients. Presumably, the **days_employed** column for this category of clients indicates the number of hours during which they were employed, and not days. In this regard, it was decided to translate the impossibly large values of hours of employment into days. For this, a threshold value of the number of days was chosen, after which the value is assumed to be impossibly large - 22265 days. This value is calculated based on the fact that in Russia a person can start his seniority at 14 years. The maximum age in this table is 75 years. (75-14) * 365 = 22265 days. This is the maximum possible number of days of employment for clients in this dataset. The **hours_to_days** function is used to convert abnormal values from hours to days.
3. Reviewing the **dob_years** column revealed that there are null values. With the help of a for loop, zero values are replaced with average values for each category of clients.
3. Using the `idxmax()` method, it is found that the **days_employed** column in line 16335 stores the value 18388, 9 days of employment (50.3 years). The age of the client is 61 years. The beginning of the seniority at 11 years is impossible. Created a **reduction** formula that identifies such anomalies and replaces the number of days of employment in the anomalies by the maximum theoretical estimated number (difference of age from the **dob_years** column and 14 years multiplied by 365 days).
4. Now that all the anomalies have been corrected, you can start calculating values that you can then fill in the gaps with. It is proposed to calculate the average value of days of service for each category of clients (column **income_type**) and replace gaps in the column **days_employed** with them. After replacing the anomalies, replacing the data in this column with the average of each customer category is appropriate. Further, it is proposed to calculate the median values of the total income for each category of citizens and replace the gaps in the **total_income** column with them. The median value is resistant to outliers possible in this column.

### Data type change

In [4]:
# Let's change the data type of the days_employed 
# and total_income columns to int64.
data = data.astype({'days_employed': 'int', 'total_income': 'int'})

print('\033[1m' + 'Table 3. The days_employed and total_income columns' +
     ' after data type changing') 
display(data[['days_employed', 'total_income']])

Table 3. The days_employed and total_income columns after data type changing


,days_employed,total_income
0,8437,253875
1,4024,112080
2,5623,145885
3,4124,267628
4,14177,158616
...,...,...
21520,4529,224791
21521,14330,155999
21522,2113,89672
21523,3112,244093


**Conclusion**

The values in the **days_employed** and **total_income** columns have the `float64` type. It is more expedient to store them in the `int64` type (displaying the number of days and money with 5 decimal places is inappropriate).
The best way to convert values to another data type in this case is `astype()`.
As a result of applying this method, the values in the above columns are in the `int64` type.

### Duplicates processing

In [5]:
# Check the dataset for duplicated rows.
print('The number of duplicated rows in the table is',
     data.duplicated().sum())

# Drop the duplicates.
data = data.drop_duplicates()

print('The number of duplicated rows in the table after removing duplicates:',
     data.duplicated().sum())

# We presume that duplicates can be found in the 
# education, education_id, family_status,
# family_status_id, gender, income_type, debt coulumns. 
# Let's check it using value_counts() method.

columns = ['education','education_id','family_status', 
           'family_status_id', 'gender', 'income_type', 'debt']

for n, column in zip(range(4,11), columns):
    print('\033[1m' + f'Table {n}. Checking column {column}' +
         ' for duplicates using value_counts()') 
    display(data[column].value_counts())
    
# The analysis showed that there are duplicates 
# in the education column.
# There is also a row in the gender column with the value XNA.
# Let's attribute it to the male gender, 
# as the least represented in the sample.
data['gender'] = data['gender'].replace('XNA', 'M')

# Convert uppercase letters to lowercase in the education column.
data['education'] = data['education'].str.lower()

The number of duplicated rows in the table is 54
The number of duplicated rows in the table after removing duplicates: 0
Table 4. Checking column education for duplicates using value_counts()


среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Table 5. Checking column education_id for duplicates using value_counts()


1    15188
0     5251
2      744
3      282
4        6
Name: education_id, dtype: int64

Table 6. Checking column family_status for duplicates using value_counts()


...

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Table 9. Checking column income_type for duplicates using value_counts()


сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Table 10. Checking column debt for duplicates using value_counts()


0    19730
1     1741
Name: debt, dtype: int64

**Conclusion**

Проведена обработка дубликатов. Удалены полные дубликаты.  
Теоретически появление дубликатов возможно в столбцах:  
* education 
* education_id
* family_status
* family_status_id
* gender
* income_type
* debt 

Методом `value_counts()` определен столбец, имеющий дубликаты. Им оказался **education**. В столбце **gender** определено значение `XNA` не имеющее смысла для исследования.  
Принято решение отнести его к мужскому полу, как к наименее представленному в выборке (при помощи метода `replace()`).  
Вероятной причиной появления дубликтов в столбце **education** является отсутствие в банке правил внесения сотрудниками банка исходной информации о его клиентах или нарушение таковых. В данном столбце строки приведены к нижнему регистру при помощи метода `srt.lower()`. В результате применения метода в столбце **education** дубликатов больше нет.

We processed the duplicates. Removed the duplicated rows.
Theoretically, the appearance of duplicates is possible in columns:
* education
* education_id
* family_status
* family_status_id
* gender
* income_type
* debt

The `value_counts()` method defines a column that has duplicates. It turned out to be **education**. The **gender** column has a value of `XNA` that is not meaningful for research.
It was decided to attribute it to the male gender, as the least represented in the sample (using the `replace()` method).
The probable reason for the appearance of duplicates in the **education** column is the lack of rules in the bank for entering initial information about its clients by bank employees or violation of such rules. Strings in this column are converted to lowercase using the `srt.lower()` method. As a result of applying the method, there are no more duplicates in the **education** column.

### Lemmatization

In [6]:
purposes = data['purpose']

# To categorize loan purposes lemmatization
# is performed in the purpose column.
# The results are listed in lemmas. The PyMyStem library is used.
m = Mystem()   
lemmas = []
for row in purposes:
    lemma = m.lemmatize(row)
    lemmas.append(lemma)

lemmas

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['покупка', ' ', 'жилье', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['недвижимость', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'п

**Conclusion**

For the categorization of loan purposes, lemmatization was carried out in the **purpose** column using the **PyMyStem** library. Lemmatization results are listed in **lemmas**. All purposes of the loan can be reduced to five lemmas (categories):
* housing (жилье)
* car (автомобиль)
* wedding (свадьба)
* education (образование)
* real estate (недвижимость)

This will be the category dictionary.

### Data Categorization

Let's create a list with categories corresponding to each lemma.  
0 - housing (жилье)  
1 - car (автомобиль)  
2 - wedding (свадьба)  
3 - education (образование)  
4 - real estate (недвижимость)

In [7]:
categories = []
for string in lemmas:
    if 'жилье' in string:
        categories.append(0)
    if 'автомобиль' in string:
        categories.append(1)
    if 'свадьба' in string:
        categories.append(2)
    if 'образование' in string:
        categories.append(3)
    if 'недвижимость'in string:
        categories.append(4)
# Add the list to the dataset.      
data['purpose_category'] = categories

print('\033[1m' + 'Table 11. The dataset after adding the ' +
     'purpose_category column') 
display(data.head(10))

Table 11. The dataset after adding the purpose_category column


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,2
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0


**Conclusion**

We have created a list of categories **categories** corresponding to each lemma. This list is added to the **data** table, following the **purpose** column.

## Hypothesis testing

- Is there a relationship between having children and repaying the loan on time?

In [8]:
# Let's change the number of children from 20 to 5 
# (20 children in a family is an anomaly).
data['children'] = data['children'].replace(20, 5)
# Find out the proportion of clients who did not repay 
# their debt on time of the total number of clients, 
# broken down by the number of children.
# Create a function.
def get_ratio_debt(dataset, column_name):
    ratio = dataset.groupby(column_name)['debt'].mean()
    return ratio
    
part_by_children = get_ratio_debt(data, 'children')
display(part_by_children)

children
0    0.075353
1    0.091639
2    0.094542
3    0.081818
4    0.097561
5    0.094118
Name: debt, dtype: float64

**Conclusion**

It is worth noting that during the construction of this table an anomaly was found in the children column.
76 clients have 20 children in the family.
It was decided to classify these cases in the category of "5 children in the family", as the closest "size" category.
The calculation of the proportion of clients who did not repay their debt on time out of the total number of clients, broken down by the number of children in the family, confirms the existence of a relationship between the presence of children and the repayment of a loan on time among these clients.
Clients without children are more likely to repay loans on time than clients with children by 1.5%. Having examined clients who have children, it was not possible to draw a clear conclusion about the relationship between the number of children and the repayment of the loan on time. In about 9% of cases, clients with children do not repay the loan on time.

- Is there a relationship between marital status and loan repayment on time?

In [9]:
# Calculate the share of clients who did not repay 
# the loan on time, disaggregated by marital status.
# For clarity, the sort_values() sorting method is used.
part_by_family_status = get_ratio_debt(data, 'family_status')
display(part_by_family_status.sort_values(ascending=False))

family_status
Не женат / не замужем    0.097509
гражданский брак         0.093202
женат / замужем          0.075421
в разводе                0.071130
вдовец / вдова           0.065693
Name: debt, dtype: float64

**Conclusion**

The calculation of the proportion of clients who did not repay the loan on time out of the total number of clients, broken down by marital status, confirms the existence of a relationship between marital status and loan repayment on time among these clients.  
Clients who have not formalized their relationship more often do not repay the loan on time (9-10%) than clients who are married or have had experience of married life (7-8%).  
It is worth noting that when compiling the types of marital status, the bank misinterpreted the concept of civil marriage.  
The categories "civil marriage" and "married / married" are one and the same. The bank implies that a civil marriage is the cohabitation of people who are in a relationship, without official registration of relations with the authorities. So it was worth calling this category - "cohabitation".

- Is there a relationship between the level of income and repayment of the loan on time?

In [10]:
# To identify the dependence, it is necessary to divide 
# the income of clients into several groups and categorize. 
# Let's create a function:
def income_group(total_income):
    if total_income <= 36000:
        return 'less than average salary'
    if total_income <= 97000:
        return 'above than average salary'
    if total_income <= 136000:
        return 'middle class'
    return 'above the middle class'

data['income_group'] = data['total_income'].apply(income_group)

# Calculate the proportion of clients who did not repay 
# the loan on time, by income category.
# For clarity, the sort_values() sorting method is used.
part_by_income_group = get_ratio_debt(data, 'income_group')
display(part_by_income_group.sort_values(ascending=False))

income_group
middle class                 0.084161
above than average salary    0.080585
above the middle class       0.080196
less than average salary     0.053333
Name: debt, dtype: float64

**Conclusion**

In order to correctly determine the presence of a relationship between the level of income and repayment of the loan on time, the categorization of clients' incomes was carried out.  
Based on the average monthly per capita income of the Russian Federation in 2020 according to Rosstat (approximately 36,000 rubles).  
Further, the categories "above than average salary", "middle class" and "above middle class" were speculatively created. It is worth noting that the concept of the middle class has many different interpretations, and, of course, does not have an exact limit in the amount of the average monthly income of a person.
The calculation of the proportion of clients who did not repay their debt on time out of the total number of clients by income category confirms the existence of a relationship between the level of income and repayment of the loan on time among these clients.
Thus, representatives of the categories "middle class", "above the middle class", "above average salary" repay the loan with violation of terms by about 3% more often than representatives of the category "less than average salary".

- How do different purposes of the loan affect its repayment on time?

Remind the categories:  
0 - housing (жилье)  
1 - car (автомобиль)  
2 - wedding (свадьба)  
3 - education (образование)  
4 - real estate (недвижимость)

In [11]:
# Calculate the proportion of customers who did not repay 
# the loan on time, broken down by category of purpose 
# of obtaining a loan.
# For clarity, the sort_values() sorting method is used.
part_by_purpose_category = get_ratio_debt(data, 'purpose_category')
display(part_by_purpose_category.sort_values(ascending=False))

purpose_category
1    0.093547
3    0.092177
2    0.079657
4    0.074610
0    0.069043
Name: debt, dtype: float64

**Conclusion**

Calculation of the share of clients who did not repay their debt on time out of the total number of clients, broken down by categories of purposes for obtaining a loan, confirms the existence of a relationship between the objectives of the loan and repayment of the loan on time among these clients.  
Thus, in the categories "car" and "education" client do not repay the loan on time in about 9% of cases.  
In the "wedding" and "real estate" categories, cleints do not repay the loan on time in about 8% of cases.  
In the "housing" category, clients do not repay the loan on time the least - in 7% of cases.

## General conclusion

Statistics analysis on the solvency of customers made it possible to make 3 statements:
1. Clients without children are more likely to repay the loan on time than clients with children. The number of children does not affect the fulfillment of the loan obligation.
2. Clients who have not formalized their relationship are more likely to violate the loan repayment terms than clients who are married or have previously had married life.
3. Clients who have taken out a loan for housing needs are more likely to repay it on time. Least of all, loans taken for needs related to a car and education are returned on time.